In [9]:
import psycopg2
import os
from dotenv import load_dotenv
from datetime import datetime
import pandas as pd

load_dotenv()
try:
    conn = psycopg2.connect(
        dbname=os.getenv("DB_NAME"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        host=os.getenv("DB_HOST"),
        port=os.getenv("DB_PORT")
    )
    print("Connection established successfully!")
except Exception as e:
    print("Connection failed:", e)


cursor = conn.cursor()

cursor.execute("SELECT * FROM potholes")  # Replace with your desired query

# Fetch results (if applicable)
rows = cursor.fetchall()

df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])


Connection established successfully!


In [10]:
#Changing the first_reported_date column to just date
for index, row in df.iterrows():
    df['first_reported_date'] =  pd.to_datetime(df['first_reported_date']).dt.date

df

,pothole_id,pothole_size,first_reported_date,number_of_reports,coordinates,is_fixed,updated_at,description,address,is_reported
0,40,Small,2024-06-19,1,"43.482921333260826, -80.51461819739562",False,2024-06-19 20:22:47.807000+00:00,Left lane,"122 Macgregor Crescent, Waterloo, Ontario N2J ...",False
1,41,Medium,2024-06-19,1,"43.47555811807502, -80.51565095313065",False,2024-06-19 20:23:19.352000+00:00,Right lane,"128 Marshall Street, Waterloo, Ontario N2J 2T7...",False
2,43,Large,2024-06-19,1,"43.478928936613585, -80.52067374289663",False,2024-06-19 21:07:34.454000+00:00,Right lane,"255 Weber Street North, Waterloo, Ontario N2J ...",False
3,44,Small,2024-06-19,1,"43.48088, -80.521015",False,2024-06-19 21:13:46.687000+00:00,Left lane,"300 Weber Street North, Waterloo",False
4,45,Small,2024-07-02,1,"43.482236381246224, -80.51078069502678",False,2024-07-02 23:47:39.175000+00:00,Middle lane,"307 University Avenue East, Waterloo, Ontario ...",False
5,48,Large,2024-07-02,1,"43.41541, -80.51149",True,2024-07-02 23:51:19.262000+00:00,Left lane,"685 fischer-hallman rd, Kitchener",False
6,49,Small,2024-07-03,1,"43.48033806406887, -80.50874700559523",False,2024-07-03 00:24:27.423000+00:00,Left lane,"188 Mayfield Avenue, Waterloo, Ontario N2J 3S4...",False
7,50,Small,2024-07-03,1,"43.47714277631787, -80.50864718238206",False,2024-07-03 00:53:49.133000+00:00,Left lane,"355 hespeler road, Cambridge",False
8,42,Medium,2024-06-19,1,"43.483713466146924, -80.51872032255609",True,2024-06-19 20:23:46.808000+00:00,Left lane,"341 Marsland Drive, Waterloo, Ontario N2J 3Z2,...",False
9,51,L,2024-07-03,1,"43.393133, -80.320013",True,2024-07-03 00:54:48.184000+00:00,L,"355 hespeler road, cambridge",False
